Read only stvari

In [1]:
import os
states=[
    'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut',
    'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa',
    'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan',
    'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
    'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio',
    'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota',
    'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia',
    'Wisconsin', 'Wyoming'
]
main_folder=".."
fullset=os.path.join(main_folder,'d50States10k')

Definisemo dataset

In [2]:
import torch
import torchvision
import numpy as np
import os 
import cv2
from torch.utils.data import Dataset,TensorDataset

def obradi(slika):
    r=slika[:,:,0]
    g=slika[:,:,1]
    b=slika[:,:,2]
    slika = np.stack([r, g, b], axis=0)
    slika=slika.astype(float)
    slika/=255
    slika=torch.from_numpy(slika)
    return slika

Klasa i inicijalizacija seta 

In [3]:
class LokacijeDataset(Dataset):
    def __init__(self,tip):
        imena=[] #Ime slike
        klase=[] #Broj klase
        for i in range(0,50):
            put=os.path.join(fullset,states[i])
            slike=os.listdir(put)
            poc=0
            if tip!="train":
                poc=2250*4
            for j in range(poc,len(slike),4):
               # print(j,tip)
                if j//4>=2250 and tip=="train":
                    break
                #print(states[i],slike[j],slike[j][:-7])
                imena.append(slike[j][:-6])
                klase.append(i)
        self.klase=klase
        self.imena=imena
    def __len__(self):
        return len(self.klase)
    def __getitem__(self,idx):
        ime = self.imena[idx]
        klasa = self.klase[idx]
        #print(klasa,ime)
        ret=[]
        for i in range(0,360,90):
            #print(ime)
            trenime=ime+"_"+str(i)+".jpg"
            put=os.path.join(fullset,states[klasa])
            put=os.path.join(put,trenime)
           # print(put)
            slika=cv2.imread(put)
            slika=obradi(slika)
            ret.append(slika)
        del slika
        return { 
            'data':torch.stack(ret),
            'label':self.klase[idx]
        }

train_set = LokacijeDataset("train")
test_set= LokacijeDataset("test")

In [4]:
if torch.cuda.is_available():
    device="cuda"
else:
    device="cpu"
    print("NE RADI CUDA!")

Inicijalizacija mreze

In [5]:
import torch.nn as nn

class Mreza (nn.Module):
    def __init__ (self):
        super().__init__()
        self.konvolucija=nn.Sequential(
            nn.Conv2d (3, 12, 3, 1, 1), #256
            nn.BatchNorm2d(12),
            nn.ReLU(), 
            nn.MaxPool2d(2),

            nn.Conv2d (12, 24, 3, 1, 1),#128
            nn.BatchNorm2d(24),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d (24, 48, 3, 1, 1),#64
            nn.BatchNorm2d(48),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d (48, 96, 3, 1, 1),#32
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d (96, 192, 3, 1, 1),#16
            nn.BatchNorm2d(192),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d (192, 384, 3, 1, 1),#8
            nn.BatchNorm2d(384),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Flatten()
        )
        self.povezan=nn.Sequential(
            nn.Linear (24576, 1000),
            nn.ReLU(),
            nn.Linear(1000, 450),
            nn.ReLU(),
            nn.Linear(450, 50),
            nn.ReLU()
        )
        self.povezan.to(device)
        self.konvolucija.to(device)

    def forward (self, x):
       # print(x.size())
        x=x.type(torch.float)  
        a=self.konvolucija(x[:,0,:,:,:])
        b=self.konvolucija(x[:,1,:,:,:])
        c=self.konvolucija(x[:,2,:,:,:])
        d=self.konvolucija(x[:,3,:,:,:])
        tens=torch.cat([a,b,c,d],1)
        del a,b,c,d
        return self.povezan(tens)
model = Mreza()

Testiranje

In [13]:
def totensor(y):
    ret=np.zeros((len(y),50))
   # print(y[0])
   # print(y[0][0])
   # print(y[0][0].item())
    #print(y)
    for i in range(0,len(y)):
        ret[i][round(y[i].item())]=1
    ret= torch.Tensor(ret)
    ret=ret.to(device)
    return ret

def get_accuracy(y_hat,y):
    #print(y_hat,y)
    y_hat = y_hat.cpu().detach().numpy()
    y=y.cpu().numpy()
    y_hat=np.argmax(y_hat,axis=1)
    corrects=(y_hat==y).sum()
    accuracy=100.0*corrects/y_hat.shape[0]
    return accuracy.item()

def test(model,sample):
    #print(sample)
    sample.to(device)
    rez=model(sample.reshape(1,4,3,256,256))
    #print(rez)
    rez=rez.detach().numpy()
    #print(rez)
    poz=0
    for i in range(1,len(rez)):
        if(rez[i]>rez[poz]):
            poz=i
    return poz

def validation(model):
    #model.to("cpu")
    kol=0
    for i in range(0,len(test_set)):
        out=test(model,test_set[i][0])
        ans=test_set[i][1][0].item()
        if round(ans)==out:
            kol+=1
    
    return kol

Treniranje

In [16]:
broj_epoha = 20
batch_velicina=100
loader = torch.utils.data.DataLoader(dataset=train_set,batch_size=batch_velicina,shuffle=True,num_workers=0,drop_last=True,pin_memory=True)

loss_funk = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

for epoha in range(1,broj_epoha+1):#Preporucujem da negde stavis koliko treba do kraja - Nemanja
    train_running_loss =0.0
    train_acc=0.0
    model = model.train()

    for batch_num, lokacije in enumerate(loader):
           # print("RADI")
            x=lokacije['data']
            y=lokacije['label']
            x = x.to(device)
            y = y.to(device)
           # print(x)
            y_hat = model(x)
            #print(y_hat.get_device())
            #print(y.get_device())
            loss = loss_funk(y_hat, totensor(y))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()


            train_running_loss += loss.detach().item()
            train_acc += get_accuracy(y_hat=y_hat, y=y)
            del y_hat
            del x
            del y 
            del loss

        
    epoch_loss = train_running_loss / batch_num
    epoch_acc = train_acc / batch_num
        
    print('Epoch: %d | Loss: %.4f | Train Accuracy: %.2f | Validation Accuracy: %.2f' \
          %(epoha, epoch_loss, epoch_acc, validation(model)/len(test_set)*100))

KeyboardInterrupt: 

In [ ]:
a=test_set[0]['data']
a=a.reshape(1,4,3,256,256)
print(a)
model(a)

tensor([[[[[0.6431, 0.6392, 0.6314,  ..., 0.6000, 0.6000, 0.6000],
           [0.6431, 0.6392, 0.6314,  ..., 0.6000, 0.6000, 0.6000],
           [0.6392, 0.6353, 0.6314,  ..., 0.6000, 0.6000, 0.6000],
           ...,
           [0.2588, 0.2353, 0.2745,  ..., 0.0784, 0.1569, 0.0863],
           [0.2196, 0.1961, 0.2745,  ..., 0.1176, 0.0627, 0.1608],
           [0.2706, 0.1059, 0.2196,  ..., 0.1020, 0.1647, 0.0745]],

          [[0.4863, 0.4824, 0.4745,  ..., 0.4275, 0.4275, 0.4275],
           [0.4863, 0.4824, 0.4745,  ..., 0.4275, 0.4275, 0.4275],
           [0.4824, 0.4784, 0.4745,  ..., 0.4275, 0.4275, 0.4275],
           ...,
           [0.2902, 0.2784, 0.3255,  ..., 0.0902, 0.1686, 0.0980],
           [0.2471, 0.2314, 0.3255,  ..., 0.1294, 0.0745, 0.1725],
           [0.2980, 0.1412, 0.2627,  ..., 0.1137, 0.1765, 0.0863]],

          [[0.3216, 0.3176, 0.3098,  ..., 0.2824, 0.2824, 0.2824],
           [0.3216, 0.3176, 0.3098,  ..., 0.2824, 0.2824, 0.2824],
           [0.3176, 0.3137

tensor([[0.2328, 0.0000, 0.0000, 0.0354, 0.0538, 0.0000, 0.0035, 0.0000, 0.0000,
         0.0467, 0.2303, 0.0000, 0.0000, 0.0956, 0.0212, 0.0000, 0.0538, 0.0000,
         0.0000, 0.0000, 0.0731, 0.0000, 0.0000, 0.0000, 0.1463, 0.0000, 0.1336,
         0.0000, 0.0000, 0.0000, 0.0241, 0.0000, 0.2073, 0.0273, 0.1420, 0.1940,
         0.0000, 0.0000, 0.0000, 0.0955, 0.0000, 0.0000, 0.0000, 0.0000, 0.0476,
         0.0000, 0.0000, 0.1352, 0.0000, 0.0942]], grad_fn=<ReluBackward0>)

In [7]:
for i in range(0,len(test_set)):
    a=test_set[i]['data']
    if(test(model,a)==test_set[i]['label']):
        print(i)
#print(test(model,test_set[2000]['data']),test_set[2000]['label'])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249


KeyboardInterrupt: 